<a href="https://colab.research.google.com/github/kotech1/computervision/blob/master/07_cnn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

model = VGG16(weights='imagenet', include_top=True)

img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02110063', 'malamute', 0.3237268), ('n02110185', 'Siberian_husky', 0.21747053), ('n02109961', 'Eskimo_dog', 0.15270932)]


In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow import keras
import numpy as np

SAVE_DIR = '/mydrive'
model = VGG16(weights=None, classes=10)
try:
  model.load_weights(SAVE_DIR+'/best_model.h5')
  print('load done')
except:
  pass
#print(model.summary())
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# create a data generator
datagen = ImageDataGenerator()

DATA_DIR = 'imagenette2-320'

# load and iterate training dataset
train_it = datagen.flow_from_directory(DATA_DIR+'/train/', target_size=(224, 224), class_mode='categorical', batch_size=64)
# load and iterate validation dataset
val_it = datagen.flow_from_directory(DATA_DIR+'/val/', target_size=(224, 224), class_mode='categorical', batch_size=64)
# load and iterate test dataset
#test_it = datagen.flow_from_directory(DATA_DIR+'/test/', class_mode='binary', batch_size=64)

checkpoint = ModelCheckpoint(SAVE_DIR+"/best_model.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', period=1)

from tensorflow.keras.callbacks import Callback

class LrReducer(Callback):
    def __init__(self, patience=0, reduce_rate=0.5, reduce_nb=10, verbose=1):
        super(Callback, self).__init__()
        self.patience = patience
        self.wait = 0
        self.best_score = -1.
        self.reduce_rate = reduce_rate
        self.current_reduce_nb = 0
        self.reduce_nb = reduce_nb
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        #current_score = logs.get('val_accuracy')
        current_score = logs.get('accuracy')
        if current_score > self.best_score:
            self.best_score = current_score
            self.wait = 0
            if self.verbose > 0:
                print('---current best score: %.3f' % current_score)
        else:
            if self.wait >= self.patience:
                self.current_reduce_nb += 1
                if self.current_reduce_nb <= 10:
                    lr = keras.backend.get_value(self.model.optimizer.lr)
                    keras.backend.set_value(self.model.optimizer.lr, lr*self.reduce_rate)
                    if self.verbose > 0:
                        print('---lr decreasing: %.3f' % (lr*self.reduce_rate))
                else:
                    if self.verbose > 0:
                        print("Epoch %d: early stopping" % (epoch))
                    self.model.stop_training = True
            self.wait += 1
lrreducer = LrReducer(patience=5)
# fit model
#model.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=8)
history = model.fit(train_it, steps_per_epoch=16, validation_data=val_it, 
          validation_steps=8, 
          epochs=100, callbacks=[checkpoint,lrreducer])


load done
Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.
Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 0.3328 - accuracy: 0.9023
Epoch 00001: loss improved from inf to 0.33279, saving model to /mydrive/best_model.h5
---current best score: 0.902
16/16 [==============================] - 37s 2s/step - loss: 0.3328 - accuracy: 0.9023 - val_loss: 2.0553 - val_accuracy: 0.5195
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 0.2978 - accuracy: 0.8975
Epoch 00002: loss improved from 0.33279 to 0.29782, saving model to /mydrive/best_model.h5
16/16 [==============================] - 36s 2s/step - loss: 0.2978 - accuracy: 0.8975 - val_loss: 2.6532 - val_accuracy: 0.5410
Epoch 3/100
16/16 [==============================] - ETA: 0s - loss: 0.2541 - accuracy: 0.9209
Epoch 00003: loss improved from 0.29782 to 0.25413, saving model to /mydrive/best_model.h5
---current best score: 0.921
16/16 [===========================

TypeError: ignored

In [6]:
history.history

{'accuracy': [0.8916015625,
  0.8818359375,
  0.9140625,
  0.931640625,
  0.9404296875,
  0.9482421875,
  0.9521484375,
  0.9599609375,
  0.9501953125,
  0.9501953125,
  0.953125,
  0.9560546875,
  0.9541015625],
 'loss': [0.3450586497783661,
  0.38263922929763794,
  0.28850334882736206,
  0.2326890379190445,
  0.20077776908874512,
  0.1876901239156723,
  0.17775705456733704,
  0.1484491229057312,
  0.17039915919303894,
  0.16273608803749084,
  0.15368404984474182,
  0.14832033216953278,
  0.1632329225540161],
 'val_accuracy': [0.564453125,
  0.5390625,
  0.513671875,
  0.546875,
  0.587890625,
  0.529296875,
  0.548828125,
  0.580078125,
  0.5625,
  0.556640625,
  0.572265625,
  0.576171875,
  0.5390625],
 'val_loss': [2.0568630695343018,
  2.096158981323242,
  2.2784550189971924,
  2.3080766201019287,
  1.9001349210739136,
  2.29915452003479,
  2.257049083709717,
  2.1322591304779053,
  2.3959884643554688,
  2.2066664695739746,
  2.2292568683624268,
  2.2070472240448,
  2.30990886688

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 01_yolo					       집.gslides
'Awesome Note'					       시사.gslides
'Colab Notebooks'				       imagenette2-320.tar.gz
'Copy of K-MOOC 한글화 용어사전.gsheet'		       Physics
'Copy of kmooc survey copied.gform'		       practice.gdoc
 data						       saved_model
 extracted_chris.zip				      'Untitled Document.gdoc'
'Google 노트에서 가져온 문서 - 내 첫 번째 노트.gdoc'   yolov3


In [19]:
!mv /mydrivebest_model.h5 /mydrive/best_model.h5

In [20]:
!ls -l /*.h5

ls: cannot access '/*.h5': No such file or directory


In [10]:
!mv best_model.h5 /mydrive


In [ ]:
!tar xfvz /mydrive/imagenette2-320.tar.gz

In [6]:
!ls

gdrive	imagenette2-320  sample_data


In [7]:
!ls -a

.  ..  .config	gdrive	imagenette2-320  sample_data


In [9]:
!ls -a /


.	 datalab     lib    mydrive		  run	 tensorflow-1.15.2
..	 dev	     lib32  mydrivebest_model.h5  sbin	 tmp
bin	 .dockerenv  lib64  opt			  srv	 tools
boot	 etc	     media  proc		  swift  usr
content  home	     mnt    root		  sys	 var


In [10]:
!ls /mydrive

 01_yolo					       집.gslides
'Awesome Note'					       시사.gslides
 best_model.h5					       imagenette2-320.tar.gz
'Colab Notebooks'				       Physics
'Copy of K-MOOC 한글화 용어사전.gsheet'		       practice.gdoc
'Copy of kmooc survey copied.gform'		       saved_model
 data						      'Untitled Document.gdoc'
 extracted_chris.zip				       yolov3
'Google 노트에서 가져온 문서 - 내 첫 번째 노트.gdoc'


In [16]:
!ls -a /root/.keras

.  ..  keras.json
